<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/36_Reproducibility_Seeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 36: Reproducibility (Walka z Chaosem)

W Deep Learningu losowość jest wszędzie:
1.  Inicjalizacja wag.
2.  Tasowanie danych (Shuffle).
3.  Dropout.
4.  **Operacje atomowe na GPU** (kolejność sumowania liczb zmiennoprzecinkowych ma znaczenie!).

Aby uzyskać **identyczny wynik** bit-w-bit przy ponownym uruchomieniu, musimy zamrozić cztery poziomy losowości:
1.  Python (`random`).
2.  NumPy (`np.random`).
3.  PyTorch CPU/GPU (`torch.manual_seed`).
4.  **CuDNN Backend** (Algorytmy splotu).

W tej lekcji stworzymy "God Function" do seedowania wszystkiego.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
import os

# Sprawdźmy sprzęt
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Urządzenie: {device}")

Urządzenie: cuda


## Eksperyment: Chaos (Bez Seeda)

Uruchom poniższą komórkę **dwa razy**.
Za każdym razem zobaczysz inne liczby. To naturalne zachowanie.

In [2]:
# Prosta sieć
net = nn.Linear(2, 2).to(device)
input_data = torch.randn(1, 2).to(device)

output = net(input_data)

print("--- WYNIK LOSOWY ---")
print(output)
print("(Uruchom tę komórkę ponownie, a wynik się zmieni)")

--- WYNIK LOSOWY ---
tensor([[ 1.0944, -0.1621]], device='cuda:0', grad_fn=<AddmmBackward0>)
(Uruchom tę komórkę ponownie, a wynik się zmieni)


## Rozwiązanie: Funkcja `seed_everything`

To jest funkcja, którą powinieneś mieć w swoim `utils.py`.

**Kluczowe flagi CuDNN:**
*   `deterministic = True`: Wymusza użycie tylko deterministycznych algorytmów splotu (np. zawsze ta sama kolejność dodawania).
*   `benchmark = False`: Zabrania CuDNN szukania najszybszego algorytmu dla danej karty graficznej (bo to szukanie wprowadza szum).

**Opcja Nuklearna:** `torch.use_deterministic_algorithms(True)`.
Jeśli ją włączysz, PyTorch **rzuci błędem**, jeśli spróbujesz użyć funkcji, która nie ma deterministycznej implementacji (np. niektóre rodzaje `scatter_add` na starych CUDA).

In [3]:
def seed_everything(seed: int = 42):
    """
    Zamraża wszystkie generatory liczb losowych.
    """
    print(f"🔒 Seeding everything with: {seed}")
    
    # 1. Python
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) # Ważne dla słowników i setów
    
    # 2. NumPy
    np.random.seed(seed)
    
    # 3. PyTorch (CPU + GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # Dla wszystkich GPU
    
    # 4. CuDNN (NVIDIA magic)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
    print("✅ Chaos opanowany.")

# Zastosujmy to
seed_everything(123)

🔒 Seeding everything with: 123
✅ Chaos opanowany.


## Weryfikacja: Stabilność

Teraz wykonamy operację, która angażuje losowość (inicjalizacja wag + dropout).
Uruchomimy ten kod dwa razy (w pętli symulującej restart skryptu).
Wyniki muszą być **identyczne**.

In [4]:
def run_simulation():
    # Model z Dropoutem (który jest losowy)
    model = nn.Sequential(
        nn.Linear(10, 10),
        nn.Dropout(0.5),
        nn.Linear(10, 1)
    ).to(device)
    
    # Losowe dane
    x = torch.randn(2, 10).to(device)
    return model(x)

print("\n--- PRÓBA 1 ---")
seed_everything(42)
result_1 = run_simulation()
print(result_1)

print("\n--- PRÓBA 2 (Restart) ---")
seed_everything(42) # Resetujemy ziarno
result_2 = run_simulation()
print(result_2)

# Sprawdzenie
diff = (result_1 - result_2).abs().sum().item()
print("-" * 30)
if diff == 0.0:
    print("✅ SUKCES: Wyniki są identyczne bit-w-bit.")
else:
    print(f"❌ PORAŻKA: Różnica wynosi {diff}")


--- PRÓBA 1 ---
🔒 Seeding everything with: 42
✅ Chaos opanowany.
tensor([[0.8472],
        [0.2100]], device='cuda:0', grad_fn=<AddmmBackward0>)

--- PRÓBA 2 (Restart) ---
🔒 Seeding everything with: 42
✅ Chaos opanowany.
tensor([[0.8472],
        [0.2100]], device='cuda:0', grad_fn=<AddmmBackward0>)
------------------------------
✅ SUKCES: Wyniki są identyczne bit-w-bit.


## 🥋 Black Belt Summary

1.  **Koszt:** Ustawienie `cudnn.deterministic = True` może spowolnić trening o **10-20%**, ponieważ wyłącza pewne optymalizacje sprzętowe.
2.  **Kiedy używać?**
    *   Podczas debugowania (musisz mieć pewność, że zmiana wyniku wynika ze zmiany kodu, a nie losowości).
    *   W publikacjach naukowych / audytach.
3.  **Kiedy NIE używać?**
    *   Podczas finalnego treningu na wielką skalę (gdy zależy Ci na każdym % szybkości), chyba że wymogi projektu stanowią inaczej.
4.  **DataLoader:** Pamiętaj, że jeśli używasz `num_workers > 0`, musisz też ustawić `worker_init_fn`, żeby każdy worker dostał inne ziarno (zazwyczaj `seed + worker_id`).